In [5]:
# Data Preparation
import os
import time
import pickle
import hashlib
import datetime

import numpy as np
import pandas as pd
import yfinance as yf
from fredapi import Fred

# Start the timer
start_time = time.time()

# %% Define stock symbol, api key, functions for retrieving stock data
symbol = 'QCOM'

# FRED API key
fred_api_key = os.getenv('FRED_API_KEY')
fred = Fred(api_key=fred_api_key)

def make_tz_naive(df):
    if df.index.tzinfo is not None:
        df.index = df.index.tz_localize(None)
    return df

def get_stock_data(symbol, start_date, end_date):
    return make_tz_naive(yf.Ticker(symbol).history(start=start_date, end=end_date))

def calculate_technical_indicators(data):
    result = data.copy()
    close = result['Close']
    result['SMA_20'] = close.rolling(window=20).mean()
    result['EMA_20'] = close.ewm(span=20, adjust=False).mean()
    result['BB_Middle'] = result['SMA_20']
    bb_std = close.rolling(window=20).std()
    result['BB_Upper'] = result['BB_Middle'] + 2 * bb_std
    result['BB_Lower'] = result['BB_Middle'] - 2 * bb_std
    delta = close.diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    result['RSI'] = 100 - (100 / (1 + rs))
    ema_12 = close.ewm(span=12, adjust=False).mean()
    ema_26 = close.ewm(span=26, adjust=False).mean()
    result['MACD'] = ema_12 - ema_26
    result['MACD_Signal'] = result['MACD'].ewm(span=9, adjust=False).mean()
    result['OBV'] = (np.sign(delta) * result['Volume']).fillna(0).cumsum()

    #? New volatility indicator: Average True Range (ATR)
    tr1 = result['High'] - result['Low']
    tr2 = abs(result['High'] - result['Close'].shift())
    tr3 = abs(result['Low'] - result['Close'].shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    result['ATR'] = tr.rolling(window=14).mean()

    return result

def get_economic_indicators(fred, start_date, end_date):
    fred_series = {
        'GDP': 'GDP', 'Interest_Rates': 'FEDFUNDS', 'Consumer_Confidence': 'UMCSENT',
        'Industrial_Production': 'INDPRO', 'Unemployment_Rate': 'UNRATE',
        'Retail_Sales': 'RSAFS', 'Housing_Starts': 'HOUST', 'Corporate_Profits': 'CP',
        'Inflation_Rate': 'CPIAUCSL', 'Economic_Policy_Uncertainty': 'USEPUINDXD'
    }
    fred_data = pd.DataFrame({name: fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
                              for name, series_id in fred_series.items()})
    return make_tz_naive(fred_data)

def get_market_indices(start_date, end_date):
    indices = yf.download(['^GSPC', '^VIX'], start=start_date, end=end_date)['Close']
    indices.columns = ['SP500', 'VIX']
    return make_tz_naive(indices)

def get_sector_data(symbol, start_date, end_date):
    sector_etfs = {
        'Information Technology': 'XLK', 'Health Care': 'XLV', 'Financials': 'XLF',
        'Consumer Discretionary': 'XLY', 'Communication Services': 'XLC',
        'Industrials': 'XLI', 'Consumer Staples': 'XLP', 'Energy': 'XLE',
        'Utilities': 'XLU', 'Real Estate': 'XLRE', 'Materials': 'XLB'
    }
    stock = yf.Ticker(symbol)
    sector = stock.info.get('sector', 'Unknown')  # Use 'Unknown' if sector info is not available
    sector_etf = sector_etfs.get(sector, 'SPY')
    sector_data = yf.download(sector_etf, start=start_date, end=end_date)['Close']
    
    # Ensure sector_data is a 1-dimensional Series
    sector_data = sector_data.squeeze()  # Convert to Series if it's a DataFrame with a single column
    
    # Create a DataFrame with the ETF prices
    sector_df = pd.DataFrame({
        f'{sector}_ETF': sector_data
    }, index=sector_data.index)  # Ensure the index is set correctly
    
    return make_tz_naive(sector_df)

# Data Collection
print("\nData Collection")
start_date, end_date = '2014-11-11', '2024-11-11'  # Example date range

stock_data = get_stock_data(symbol, start_date, end_date)
print(stock_data.info())
data_with_indicators = calculate_technical_indicators(stock_data)
economic_data = get_economic_indicators(fred, start_date, end_date)
market_indices = get_market_indices(start_date, end_date)
sector_data = get_sector_data(symbol, start_date, end_date)

# Prefix columns to avoid overlapping
data_with_indicators = data_with_indicators.add_prefix(f'{symbol}_')
# economic_data = economic_data.add_prefix('Economic_')
# market_indices = market_indices.add_prefix('Market_')
# sector_data = sector_data.add_prefix('Sector_')

# Join the DataFrames
combined_data = data_with_indicators.join([
    economic_data,
    market_indices,
    sector_data,
])

# Data Preprocessing nan and interpolate, weekend removal
def remove_nan(data):
    data = data.interpolate().ffill().bfill()
    data = data[data.index.dayofweek < 5]  # Remove weekends
    print("\nNAN values in data after preprocessing:\n", data.isna().sum())
    return data

# Data Preprocessing
cleaned_data = remove_nan(combined_data)


Data Collection
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2516 entries, 2014-11-11 to 2024-11-08
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          2516 non-null   float64
 1   High          2516 non-null   float64
 2   Low           2516 non-null   float64
 3   Close         2516 non-null   float64
 4   Volume        2516 non-null   int64  
 5   Dividends     2516 non-null   float64
 6   Stock Splits  2516 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 157.2 KB
None


[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed


NAN values in data after preprocessing:
 QCOM_Open                      0
QCOM_High                      0
QCOM_Low                       0
QCOM_Close                     0
QCOM_Volume                    0
QCOM_Dividends                 0
QCOM_Stock Splits              0
QCOM_SMA_20                    0
QCOM_EMA_20                    0
QCOM_BB_Middle                 0
QCOM_BB_Upper                  0
QCOM_BB_Lower                  0
QCOM_RSI                       0
QCOM_MACD                      0
QCOM_MACD_Signal               0
QCOM_OBV                       0
QCOM_ATR                       0
GDP                            0
Interest_Rates                 0
Consumer_Confidence            0
Industrial_Production          0
Unemployment_Rate              0
Retail_Sales                   0
Housing_Starts                 0
Corporate_Profits              0
Inflation_Rate                 0
Economic_Policy_Uncertainty    0
SP500                          0
VIX                            0
T

In [ ]:
# Calculate Business Cycle Indicators
from collections import defaultdict

def calculate_cycle_score(row, data):
    score = 0
    score += 1 if row[f'{symbol}_Close'] > row[f'{symbol}_SMA_20'] else -1
    score += 1 if row['GDP'] > data['GDP'].rolling(window=20).mean().loc[row.name] else -1
    score += 1 if row['Industrial_Production'] > data['Industrial_Production'].rolling(window=20).mean().loc[row.name] else -1
    score += 1 if row['Unemployment_Rate'] < data['Unemployment_Rate'].rolling(window=20).mean().loc[row.name] else -1
    score += 1 if row['Consumer_Confidence'] > data['Consumer_Confidence'].rolling(window=20).mean().loc[row.name] else -1
    score += 1 if row['Corporate_Profits'] > data['Corporate_Profits'].rolling(window=20).mean().loc[row.name] else -1
    score += 1 if row['SP500'] > data['SP500'].rolling(window=20).mean().loc[row.name] else -1
    return score

def determine_business_cycle(score):
    if score >= 5:
        return 'Expansion'
    elif 2 <= score < 5:
        return 'Peak'
    elif -2 <= score < 2:
        return 'Contraction'
    else:
        return 'Trough'

def iterative_bci_assignment(data):
    data[f'{symbol}_BCI_Score'] = data.apply(lambda row: calculate_cycle_score(row, data), axis=1)
    data[f'{symbol}_Raw_BCI'] = data[f'{symbol}_BCI_Score'].apply(determine_business_cycle)
    
    bci_periods = []
    current_period = data[f'{symbol}_Raw_BCI'].iloc[0]
    transition_count = 0
    transition_threshold = 20  # Adjust this value as needed

    for _, row in data.iterrows():
        if row[f'{symbol}_Raw_BCI'] == current_period:
            transition_count = 0
        else:
            transition_count += 1

        if transition_count >= transition_threshold:
            if current_period == 'Expansion':
                current_period = 'Peak'
            elif current_period == 'Peak':
                current_period = 'Contraction'
            elif current_period == 'Contraction':
                current_period = 'Trough'
            else:  # Trough
                current_period = 'Expansion'
            transition_count = 0

        bci_periods.append(current_period)

    return bci_periods

# Calculate Business Cycle Indicators
print("\nCalculating Business Cycle Indicator")
cleaned_data[f'{symbol}_Business_Cycle'] = iterative_bci_assignment(cleaned_data)

print("\nBusiness Cycle Indicator Distribution:")
print(cleaned_data[f'{symbol}_Business_Cycle'].value_counts(normalize=True))

# Calculate Average Timeframes for Each Cycle Phase
def calculate_average_timeframes(data):
    phase_durations = defaultdict(list)
    current_phase = data[f'{symbol}_Business_Cycle'].iloc[0]
    phase_start = data.index[0]
    
    for date, phase in zip(data.index[1:], data[f'{symbol}_Business_Cycle'].iloc[1:]):
        if phase != current_phase:
            duration = (date - phase_start).days
            phase_durations[current_phase].append(duration)
            current_phase = phase
            phase_start = date
    
    # Add the last phase
    duration = (data.index[-1] - phase_start).days
    phase_durations[current_phase].append(duration)
    
    average_durations = {phase: np.mean(durations) for phase, durations in phase_durations.items()}
    return average_durations

average_timeframes = calculate_average_timeframes(cleaned_data)
print("\nAverage Timeframes for Each Cycle Phase (in days):")
for phase, duration in average_timeframes.items():
    print(f"{phase}: {duration:.2f}")
avg_bc_length = sum(average_timeframes.values())
print(f"\nAverage Business Cycle Length: {avg_bc_length:.2f} days")

In [ ]:
# FFT Analysis, add statistical features
from scipy.fft import fft
from scipy.signal import detrend
from scipy.stats import kurtosis

def perform_enhanced_fft_analysis(data):
    print("\n--- Enhanced FFT Analysis ---")
    
    close_prices = data[f'{symbol}_Close'].values
    detrended_prices = detrend(close_prices)
    
    window = np.hanning(len(detrended_prices))
    windowed_prices = detrended_prices * window
    fft_result = fft(windowed_prices)
    frequencies = np.fft.fftfreq(len(detrended_prices), d=1)
    amplitudes = np.abs(fft_result)
    
    all_cycles = {}
    time_scales = [
        ('Long-term', 252, 1260),  # 1-5 years
        ('Medium-term', 21, 252),  # 1 month to 1 year
        ('Short-term', 2, 21)      # 2 days to 1 month
    ]
    
    for scale_name, min_period, max_period in time_scales:
        print(f"\n{scale_name} Analysis:")
        mask = (1/max_period <= np.abs(frequencies)) & (np.abs(frequencies) <= 1/min_period)
        scale_frequencies = frequencies[mask]
        scale_amplitudes = amplitudes[mask]
        
        sorted_indices = np.argsort(scale_amplitudes)[::-1]
        top_frequencies = scale_frequencies[sorted_indices[:5]]
        top_amplitudes = scale_amplitudes[sorted_indices[:5]]
        
        print("Top 5 dominant frequencies:")
        for i, (freq, amp) in enumerate(zip(top_frequencies, top_amplitudes), 1):
            period = 1 / abs(freq) if freq != 0 else np.inf
            print(f"{i}. Frequency: {freq:.6f}, Period: {period:.2f} days, Amplitude: {amp:.2f}")
            
            cycle_name = f"{scale_name.lower()}_cycle_{i}"
            all_cycles[cycle_name] = (freq, amp, period)
    
    variables_to_compare = ['SP500', 'VIX', 'Sector_Technology_ETF']
    for var in variables_to_compare:
        if var in data.columns:
            var_fft = fft(detrend(data[var].values) * window)
            var_amplitudes = np.abs(var_fft)
            correlation = np.corrcoef(amplitudes, var_amplitudes)[0, 1]
            print(f"\nFFT Amplitude correlation with {var}: {correlation:.4f}")
    
    return all_cycles

def add_statistical_features(data):
    print("\n--- Adding Statistical Features ---")
    window_sizes = [5, 10, 20, 50, 100]
    new_features = {}
    for window in window_sizes:
        new_features[f'rolling_mean_{window}'] = data[f'{symbol}_Close'].rolling(window=window, min_periods=1).mean()
        
        # For standard deviation, we need at least 2 points
        std = data[f'{symbol}_Close'].rolling(window=window, min_periods=2).std()
        new_features[f'rolling_std_{window}'] = std.bfill()
        
        # For skewness, we need at least 3 points
        skew = data[f'{symbol}_Close'].rolling(window=window, min_periods=3).skew()
        new_features[f'rolling_skew_{window}'] = skew.bfill()
        
        # For kurtosis, we need at least 4 points
        kurt = data[f'{symbol}_Close'].rolling(window=window, min_periods=4).apply(kurtosis)
        new_features[f'rolling_kurt_{window}'] = kurt.bfill()
        
        print(f"Added rolling statistics for window size {window}")
    
    result = pd.concat([data, pd.DataFrame(new_features, index=data.index)], axis=1)
    
    # Replace any remaining NaNs with the first valid value
    result = result.bfill().ffill()
    
    return result

all_cycles = perform_enhanced_fft_analysis(cleaned_data)
cleaned_data = add_statistical_features(cleaned_data)
print("\nCycle features and statistical features have been added to the dataframe.")

In [ ]:
# Check data before split and normalize
def analyze_cleaned_data(cleaned_data):
    print("\n--- Cleaned Data Analysis ---")
    print("\nData Info:")
    print(cleaned_data.info())
    
    start_date, end_date = cleaned_data.index.min(), cleaned_data.index.max()
    print(f"\nCleaned data date range: {start_date} to {end_date}")
    print(f"Total number of rows: {len(cleaned_data)}")
    print(f"Number of days between start and end date: {(end_date - start_date).days}")

    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    missing_dates = full_date_range.difference(cleaned_data.index)
    print(f"\nNumber of missing dates: {len(missing_dates)}")
    if len(missing_dates) > 0:
        print("First few missing dates:", missing_dates[:5].tolist())

    print(f"\nNumber of NaN values in '{symbol}_Close' column: {cleaned_data[f'{symbol}_Close'].isna().sum()}")

    day_counts = cleaned_data.index.dayofweek.value_counts().sort_index()
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    print("\nSummary of days of the week:")
    for day, count in day_counts.items():
        print(f"{day_names[day]}: {count}")

    weekend_data = cleaned_data[cleaned_data.index.dayofweek.isin([5, 6])]
    if not weekend_data.empty:
        print("\nWarning: Data contains weekend entries:")
        print(weekend_data)

    total_weekdays = sum(day.weekday() < 5 for day in full_date_range)
    available_weekdays = sum(day_counts[:5])
    coverage_percentage = (available_weekdays / total_weekdays) * 100
    print(f"\nPercentage of available trading days: {coverage_percentage:.2f}%")

analyze_cleaned_data(cleaned_data)


In [5]:
# Normalize data
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import torch

# Identify categorical and numeric columns
categorical_columns = cleaned_data.select_dtypes(include=['object', 'category']).columns
numeric_columns = cleaned_data.select_dtypes(include=['int64', 'float64']).columns

# Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_columns),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns)
    ])

# Fit the preprocessor on the entire dataset
preprocessor.fit(cleaned_data)

# Transform the data
data_transformed = preprocessor.transform(cleaned_data)


In [ ]:
# Perform correlation analysis on training data only
def perform_correlation_analysis(data, column_names, threshold):
    print("\n--- Correlation Analysis ---")
    
    # Convert the numpy array back to a DataFrame
    data_df = pd.DataFrame(data, columns=column_names)
    
    correlations = data_df.corr()
    
    # Correlation with target variable (assuming is the target, formatted as num__{symbol}_Close)
    # This was from the ColumnTransformer
    target_correlations = correlations[f'num__{symbol}_Close'].sort_values(ascending=False)
    target_correlations = target_correlations.drop(f'num__{symbol}_Close')
    
    print(f"\nTop 10 Positive Correlations with {symbol}_Close:")
    print(target_correlations.head(10))
    print(f"\nTop 10 Negative Correlations with {symbol}_Close:")
    print(target_correlations.tail(10))
    
    # Filter features based on correlation threshold
    strong_corr = target_correlations[abs(target_correlations) >= threshold]
    weak_corr = target_correlations[abs(target_correlations) < threshold]
    
    print(f"\nFeatures with strong correlation (|r| >= {threshold}): {len(strong_corr)}")
    print(f"Features with weak correlation (|r| < {threshold}): {len(weak_corr)}")
    
    return strong_corr, weak_corr

# Assuming you have a list of column names corresponding to the transformed data
column_names = preprocessor.get_feature_names_out()
print(column_names)
strong_corr, weak_corr = perform_correlation_analysis(data_transformed, column_names, threshold=0.5)

In [7]:
# Create sequences
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        sequences.append(seq)
    # Convert the list of sequences to a numpy array
    sequences_array = np.array(sequences)
    # Convert the numpy array to a PyTorch tensor
    return torch.FloatTensor(sequences_array)

seq_length = 60
# Get indices of strongly correlated features
strong_feature_indices = [i for i, feature in enumerate(column_names) if feature in strong_corr.index]
# Select only strongly correlated features from transformed data
strong_features_data = data_transformed[:, strong_feature_indices]

X = create_sequences(strong_features_data, seq_length)
y = torch.FloatTensor(cleaned_data[f'{symbol}_Close'].values[seq_length:])

# Split data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, shuffle=False)

In [8]:
# LSTM Model
from torch import nn, optim

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Model parameters
input_size = len(strong_feature_indices)
hidden_size = 64
num_layers = 2
output_size = 1
#! add dropout and learning rate scheduler

model = LSTMModel(input_size, hidden_size, num_layers, output_size)


In [ ]:
# Training Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Move data to GPU
X_train = X_train.to(device)
y_train = y_train.to(device)
X_val = X_val.to(device)
y_val = y_val.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)


In [ ]:
# Training Loop
num_epochs = 100
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size].view(-1, 1)  # Reshape target to match model output
        
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val.view(-1, 1))  # Reshape target to match model output
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

In [11]:
# # TensorRT-optimized LSTM model (optional)
# import torch_tensorrt

# # Compile the model with TensorRT
# trt_model = torch_tensorrt.compile(model, 
#     inputs=[torch_tensorrt.Input((batch_size, seq_length, input_size))],
#     enabled_precisions={torch.float32, torch.float16} # Enable FP32 and FP16
# )

# # Save the TensorRT optimized model
# torch.jit.save(trt_model, "trt_lstm_model.pt")

In [ ]:
# Evaluation

model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test.view(-1, 1))  # Ensure target shape matches output

print(f"Test Loss: {test_loss.item():.4f}")

# Inverse transform predictions
# Assuming the MinMaxScaler was applied only to the target variable
scaler = preprocessor.named_transformers_['num']  # Access the MinMaxScaler for numeric columns

# Inverse transform only the target variable
# Select the column index corresponding to the target variable
target_index = list(numeric_columns).index(f'{symbol}_Close')
predictions = scaler.inverse_transform(np.concatenate([np.zeros((test_outputs.shape[0], target_index)), 
                                                       test_outputs.cpu().numpy(), 
                                                       np.zeros((test_outputs.shape[0], len(numeric_columns) - target_index - 1))], axis=1))[:, target_index]
actual = scaler.inverse_transform(np.concatenate([np.zeros((y_test.shape[0], target_index)), 
                                                  y_test.cpu().numpy().reshape(-1, 1), 
                                                  np.zeros((y_test.shape[0], len(numeric_columns) - target_index - 1))], axis=1))[:, target_index]

# Calculate metrics (e.g., RMSE, MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse = np.sqrt(mean_squared_error(actual, predictions))
mae = mean_absolute_error(actual, predictions)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0  # Avoid division by zero
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100

mape = mean_absolute_percentage_error(actual, predictions)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MAPE: {mape:.2f}%")